In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# sklearn utilities
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, accuracy_score

# sklearn models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

In [21]:
county_area = pd.read_csv('county_area.csv')
county_area

,Areaname,STCOU,AREA
0,UNITED STATES,0,3787425.08
1,ALABAMA,1000,52422.94
2,Autauga AL,1001,604.49
3,Baldwin AL,1003,2027.08
4,Barbour AL,1005,904.59
...,...,...,...
3193,Sweetwater WY,56037,10491.73
3194,Teton WY,56039,4221.96
3195,Uinta WY,56041,2087.66
3196,Washakie WY,56043,2242.85


In [22]:
county_mask_use = pd.read_csv('mask_use.csv')
county_mask_use

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459
...,...,...,...,...,...,...
3137,56037,0.061,0.295,0.230,0.146,0.268
3138,56039,0.095,0.157,0.160,0.247,0.340
3139,56041,0.098,0.278,0.154,0.207,0.264
3140,56043,0.204,0.155,0.069,0.285,0.287


In [28]:
county_cases = pd.read_csv('county_cases.csv', encoding='latin-1')
county_cases

,county,state,fips,cases,deaths
0,Autauga,Alabama,1001.0,2059,31
1,Baldwin,Alabama,1003.0,6658,69
2,Barbour,Alabama,1005.0,1033,9
3,Bibb,Alabama,1007.0,840,14
4,Blount,Alabama,1009.0,1932,25
...,...,...,...,...,...
3135,Sweetwater,Wyoming,56037.0,462,2
3136,Teton,Wyoming,56039.0,747,1
3137,Uinta,Wyoming,56041.0,431,3
3138,Washakie,Wyoming,56043.0,144,7


In [30]:
county_pop = pd.read_csv('county_pop.csv', encoding='latin-1')
county_pop

,COUNTY,STNAME,CTYNAME,POPESTIMATE2019,NPOPCHG_2019,BIRTHS2019,DEATHS2019,NATURALINC2019,INTERNATIONALMIG2019,DOMESTICMIG2019,NETMIG2019,RESIDUAL2019,GQESTIMATES2019,RBIRTH2019,RDEATH2019,RNATURALINC2019,RINTERNATIONALMIG2019,RDOMESTICMIG2019,RNETMIG2019
0,0,Alabama,Alabama,4903185,15504,57313,53879,3434,2772,9387,12159,-89,116625,11.707442,11.005972,0.701470,0.566242,1.917501,2.483744
1,1,Alabama,Autauga County,55869,336,624,541,83,-16,270,254,-1,455,11.202671,9.712572,1.490099,-0.287248,4.847310,4.560062
2,3,Alabama,Baldwin County,223234,5379,2304,2326,-22,80,5297,5377,24,2170,10.446871,10.546624,-0.099753,0.362739,24.017829,24.380567
3,5,Alabama,Barbour County,24686,-186,256,312,-56,13,-141,-128,-2,2812,10.331329,12.591307,-2.259978,0.524638,-5.690302,-5.165664
4,7,Alabama,Bibb County,22394,27,240,252,-12,10,31,41,-2,2148,10.723621,11.259802,-0.536181,0.446818,1.385134,1.831952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,37,Wyoming,Sweetwater County,42343,-515,506,298,208,23,-750,-727,4,627,11.877795,6.995223,4.882572,0.539900,-17.605427,-17.065527
3189,39,Wyoming,Teton County,23464,195,228,75,153,79,-37,42,0,269,9.757559,3.209723,6.547836,3.380909,-1.583464,1.797445
3190,41,Wyoming,Uinta County,20226,-66,255,136,119,0,-186,-186,1,231,12.586998,6.713066,5.873933,0.000000,-9.181105,-9.181105
3191,43,Wyoming,Washakie County,7805,-72,83,90,-7,-5,-60,-65,0,140,10.585385,11.478128,-0.892743,-0.637674,-7.652085,-8.289759
